#Setting

https://pseudo-lab.github.io/klue-baseline/docs/RE-1.html

In [2]:
!pip install transformers==4.17 datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
from datasets import load_dataset

dataset = load_dataset('klue', 're')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32470 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7765 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 32470
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 7765
    })
})

# Fine-tune a pretrained model

## Prepare a dataset

In [4]:
re_feature = dataset["train"].features['label']
re_feature

ClassLabel(names=['no_relation', 'org:dissolved', 'org:founded', 'org:place_of_headquarters', 'org:alternate_names', 'org:member_of', 'org:members', 'org:political/religious_affiliation', 'org:product', 'org:founded_by', 'org:top_members/employees', 'org:number_of_employees/members', 'per:date_of_birth', 'per:date_of_death', 'per:place_of_birth', 'per:place_of_death', 'per:place_of_residence', 'per:origin', 'per:employee_of', 'per:schools_attended', 'per:alternate_names', 'per:parents', 'per:children', 'per:siblings', 'per:spouse', 'per:other_family', 'per:colleagues', 'per:product', 'per:religion', 'per:title'], id=None)

In [5]:
dataset["train"][0]

{'guid': 'klue-re-v1_train_00000',
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': {'word': '비틀즈',
  'start_idx': 24,
  'end_idx': 26,
  'type': 'ORG'},
 'object_entity': {'word': '조지 해리슨',
  'start_idx': 13,
  'end_idx': 18,
  'type': 'PER'},
 'label': 0,
 'source': 'wikipedia'}

Type Tokens를 추가하는 이유

추가한 token은 transformer 전체층을 다 거치고 나면 sequence의 결합된 의미를 가지게 되는데, 여기에 간단한 classifier를 붙이면 단일 문장의 classification을 쉽게 할 수 있게 됩니다.

즉 문장을 분류함에 있어서 제시된 entity들이 특별한 의미를 갖도록 하기 위해 type tokens들을 추가하는것!

Type token 추가

In [6]:
def modify_sentence(data):
    sentence = data['sentence']
    object_start =  int(data['object_entity']['start_idx'])
    object_end =  int(data['object_entity']['end_idx'])
    subject_start =  int(data['subject_entity']['start_idx'])
    subject_end =  int(data['subject_entity']['end_idx'])
    otype = data['object_entity']['type']
    stype = data['subject_entity']['type']

    if object_start < subject_start:
      new_sentence = sentence[:object_start] + '<O-' + str(otype) + '>' + sentence[object_start:object_end+1] + '</O-' + str(otype)+'>' + sentence[object_end+1:subject_start] + '<S-'+str(stype)+'>' + sentence[subject_start:subject_end+1] + '</S-'+str(stype)+'>' + sentence[subject_end+1:]
    else:
      new_sentence = sentence[:subject_start] + '<S-'+str(stype)+'>' + sentence[subject_start:subject_end+1] + '</S-'+str(stype)+'>' + sentence[subject_end+1:object_start] + '<O-'+str(otype)+'>' + sentence[object_start:object_end+1] + '</O-'+str(otype)+'>' + sentence[object_end+1:]

    data['sentence'] = new_sentence
    return data

# Modify sentences in the 'train' set
dataset['train'] = dataset['train'].map(modify_sentence)

# Modify sentences in the 'validation' set
dataset['validation'] = dataset['validation'].map(modify_sentence)

# Print the updated dataset
print(dataset)

Map:   0%|          | 0/32470 [00:00<?, ? examples/s]

Map:   0%|          | 0/7765 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 32470
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 7765
    })
})


In [7]:
dataset['train']['sentence'][:5]

['〈Something〉는 <O-PER>조지 해리슨</O-PER>이 쓰고 <S-ORG>비틀즈</S-ORG>가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 '호남이 기반인 바른미래당·<O-ORG>대안신당</O-ORG>·<S-ORG>민주평화당</S-ORG>이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.',
 'K리그2에서 성적 1위를 달리고 있는 <S-ORG>광주FC</S-ORG>는 지난 26일 <O-ORG>한국프로축구연맹</O-ORG>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.',
 '균일가 생활용품점 (주)<S-ORG>아성다이소</S-ORG>(대표 <O-PER>박정부</O-PER>)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.',
 '<O-DAT>1967</O-DAT>년 프로 야구 드래프트 1순위로 <S-ORG>요미우리 자이언츠</S-ORG>에게 입단하면서 등번호는 8번으로 배정되었다.']

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [9]:
#train
types = []
for i in range(32470):
    if dataset['train'][i]['subject_entity']['type'] not in types:
        types.append(dataset['train'][i]['subject_entity']['type'])

    if dataset['train'][i]['object_entity']['type'] not in types:
        types.append(dataset['train'][i]['object_entity']['type'])

#validation
for i in range(7765):
    if dataset['validation'][i]['subject_entity']['type'] not in types:
        types.append(dataset['validation'][i]['subject_entity']['type'])

    if dataset['validation'][i]['object_entity']['type'] not in types:
        types.append(dataset['validation'][i]['object_entity']['type'])

entity_types = []

for i in types:
    entity_types.append('S-' + str(i))
    entity_types.append('O-' + str(i))

entity_types

['S-ORG',
 'O-ORG',
 'S-PER',
 'O-PER',
 'S-DAT',
 'O-DAT',
 'S-LOC',
 'O-LOC',
 'S-POH',
 'O-POH',
 'S-NOH',
 'O-NOH']

In [10]:
# token 추가할 떄 <> 이거 안 넣어도 되나..?
new_enrollment_tokens = {'additional_special_tokens': entity_types}
enrollment_tokens = tokenizer.add_special_tokens(new_enrollment_tokens)

In [11]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/32470 [00:00<?, ? examples/s]

Map:   0%|          | 0/7765 [00:00<?, ? examples/s]

In [12]:
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_tokens)

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['S-ORG', 'O-ORG', 'S-PER', 'O-PER', 'S-DAT', 'O-DAT', 'S-LOC', 'O-LOC', 'S-POH', 'O-POH', 'S-NOH', 'O-NOH']}
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', 'S-ORG', 'O-ORG', 'S-PER', 'O-PER', 'S-DAT', 'O-DAT', 'S-LOC', 'O-LOC', 'S-POH', 'O-POH', 'S-NOH', 'O-NOH']


In [13]:
#이미 tokenize 된 경우 is_split_into_words = True 로 설정
inputs = tokenizer(dataset["train"][0]["sentence"])
print(inputs.tokens())

['[CLS]', '〈', 'So', '##me', '##th', '##ing', '〉', '는', '<', 'O-PER', '>', '조지', '해리', '##슨', '<', '/', 'O-PER', '>', '이', '쓰', '##고', '<', 'S-ORG', '>', '비틀즈', '<', '/', 'S-ORG', '>', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.', '[SEP]']


In [14]:
dataset['train'][0]

{'guid': 'klue-re-v1_train_00000',
 'sentence': '〈Something〉는 <O-PER>조지 해리슨</O-PER>이 쓰고 <S-ORG>비틀즈</S-ORG>가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': {'word': '비틀즈',
  'start_idx': 24,
  'end_idx': 26,
  'type': 'ORG'},
 'object_entity': {'word': '조지 해리슨',
  'start_idx': 13,
  'end_idx': 18,
  'type': 'PER'},
 'label': 0,
 'source': 'wikipedia'}

In [15]:
deocoded_sequence = tokenizer.decode(tokenized_datasets['train']['input_ids'][0])
print(deocoded_sequence)

[CLS] 〈 Something 〉 는 < O-PER > 조지 해리슨 < / O-PER > 이 쓰고 < S-ORG > 비틀즈 < / S-ORG > 가 1969년 앨범 《 Abbey Road 》 에 담은 노래다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

metric

In [16]:
import numpy as np
import evaluate

metric = evaluate.load('f1', 'micro')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='micro')

In [17]:
label_names = re_feature.names
print(label_names)

['no_relation', 'org:dissolved', 'org:founded', 'org:place_of_headquarters', 'org:alternate_names', 'org:member_of', 'org:members', 'org:political/religious_affiliation', 'org:product', 'org:founded_by', 'org:top_members/employees', 'org:number_of_employees/members', 'per:date_of_birth', 'per:date_of_death', 'per:place_of_birth', 'per:place_of_death', 'per:place_of_residence', 'per:origin', 'per:employee_of', 'per:schools_attended', 'per:alternate_names', 'per:parents', 'per:children', 'per:siblings', 'per:spouse', 'per:other_family', 'per:colleagues', 'per:product', 'per:religion', 'per:title']


In [18]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
print(id2label, label2id)

{0: 'no_relation', 1: 'org:dissolved', 2: 'org:founded', 3: 'org:place_of_headquarters', 4: 'org:alternate_names', 5: 'org:member_of', 6: 'org:members', 7: 'org:political/religious_affiliation', 8: 'org:product', 9: 'org:founded_by', 10: 'org:top_members/employees', 11: 'org:number_of_employees/members', 12: 'per:date_of_birth', 13: 'per:date_of_death', 14: 'per:place_of_birth', 15: 'per:place_of_death', 16: 'per:place_of_residence', 17: 'per:origin', 18: 'per:employee_of', 19: 'per:schools_attended', 20: 'per:alternate_names', 21: 'per:parents', 22: 'per:children', 23: 'per:siblings', 24: 'per:spouse', 25: 'per:other_family', 26: 'per:colleagues', 27: 'per:product', 28: 'per:religion', 29: 'per:title'} {'no_relation': 0, 'org:dissolved': 1, 'org:founded': 2, 'org:place_of_headquarters': 3, 'org:alternate_names': 4, 'org:member_of': 5, 'org:members': 6, 'org:political/religious_affiliation': 7, 'org:product': 8, 'org:founded_by': 9, 'org:top_members/employees': 10, 'org:number_of_emplo

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base",
    num_labels=30,
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [20]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32012, 768)

## Train

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "klue-bert-base-finetuned-re",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [22]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: subject_entity, guid, object_entity, sentence, source. If subject_entity, guid, object_entity, sentence, source are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


Epoch,Training Loss,Validation Loss,F1
1,1.687000,1.384099,0.549000
2,0.977700,1.138914,0.627000
3,0.698300,1.184292,0.622200


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: subject_entity, guid, object_entity, sentence, source. If subject_entity, guid, object_entity, sentence, source are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to klue-bert-base-finetuned-re/checkpoint-625
Configuration saved in klue-bert-base-finetuned-re/checkpoint-625/config.json
Model weights saved in klue-bert-base-finetuned-re/checkpoint-625/pytorch_model.bin
tokenizer config file saved in klue-bert-base-finetuned-re/checkpoint-625/tokenizer_config.json
Special tokens file saved in klue-bert-base-finetuned-re/checkpoint-625/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: su

TrainOutput(global_step=1875, training_loss=1.017114404296875, metrics={'train_runtime': 1924.1838, 'train_samples_per_second': 7.796, 'train_steps_per_second': 0.974, 'total_flos': 3947658024960000.0, 'train_loss': 1.017114404296875, 'epoch': 3.0})

In [25]:
index = 3
print(dataset["train"][index]['sentence'])
print(dataset["train"][index]['label'])

균일가 생활용품점 (주)<S-ORG>아성다이소</S-ORG>(대표 <O-PER>박정부</O-PER>)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
10


In [27]:
#결과가 좀 이상함
from transformers import pipeline

model_checkpoint = "/content/klue-bert-base-finetuned-re/checkpoint-1875"
sentence_classifier = pipeline("text-classification", model=model_checkpoint)

sentence_classifier("아성다이소, 박정부")
# print(token_classifier)

loading configuration file /content/klue-bert-base-finetuned-re/checkpoint-1875/config.json
Model config BertConfig {
  "_name_or_path": "/content/klue-bert-base-finetuned-re/checkpoint-1875",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "no_relation",
    "1": "org:dissolved",
    "2": "org:founded",
    "3": "org:place_of_headquarters",
    "4": "org:alternate_names",
    "5": "org:member_of",
    "6": "org:members",
    "7": "org:political/religious_affiliation",
    "8": "org:product",
    "9": "org:founded_by",
    "10": "org:top_members/employees",
    "11": "org:number_of_employees/members",
    "12": "per:date_of_birth",
    "13": "per:date_of_death",
    "14": "per:place_of_birth",
    "15": "per:place_of_death",
    "16": "per:place_of_residence",
    "17": "per:origin",
    "18": "per:employ

[{'label': 'no_relation', 'score': 0.3051629662513733}]